- vector_store.ipynb
- data/소설들.pdf 들  vector store에 저장.
    - chroma, pinecone
    - metadata
        - title : 소설제목
        - author : 저자
        - full_text : 소설 전체 내용
    - Document(page_content="split된 내용", metadata={})
    - page_content는 조회할 떄 사용할 embedding vector를 생성할 때 사용될 내용.
    - LLM 모델에 전송할 context는 metadata의 full_text

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

# 1. Directory Loader 객체 생성 -> loading할 문서(들)을 지정.
loader = DirectoryLoader(
    path="./data", 
    glob=["*.pdf"],
    recursive=True,    
)

docs = loader.load()


The MIME type of 'data\\금_따는_콩밭_김유정.pdf' is "cannot open `data\\\\352\\270\\210_\\353\\224\\260\\353\\212\\224_\\354\\275\\251\\353\\260\\255_\\352\\271\\200\\354\\234\\240\\354\\240\\225.pdf' (illegal byte sequence)". This file type is not currently supported in unstructured.


The MIME type of 'data\\동백꽃_김유정.pdf' is "cannot open `data\\\\353\\217\\231\\353\\260\\261\\352\\275\\203_\\352\\271\\200\\354\\234\\240\\354\\240\\225.pdf' (illegal byte sequence)". This file type is not currently supported in unstructured.
The MIME type of 'data\\메밀꽃_필_무렵_이효석.pdf' is "cannot open `data\\\\353\\251\\224\\353\\260\\200\\352\\275\\203_\\355\\225\\204_\\353\\254\\264\\353\\240\\265_\\354\\235\\264\\355\\232\\250\\354\\204\\235.pdf' (illegal byte sequence)". This file type is not currently supported in unstructured.
The MIME type of 'data\\배따라기_김동인.pdf' is "cannot open `data\\\\353\\260\\260\\353\\224\\260\\353\\235\\274\\352\\270\\260_\\352\\271\\200\\353\\217\\231\\354\\235\\270.pdf' (illegal byte sequence)". This file type is not currently supported in unstructured.
The MIME type of 'data\\백치_아다다_계용묵.pdf' is "cannot open `data\\\\353\\260\\261\\354\\271\\230_\\354\\225\\204\\353\\213\\244\\353\\213\\244_\\352\\263\\204\\354\\232\\251\\353\\254\\265.pdf' (illegal byte s

In [15]:
# 2. RecursiveCharacterTextSplitter 객체 생성 -> Document를 분할.
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100)

split_docs = splitter.split_documents(docs)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0)

docs = loader.load_and_split(splitter)

The MIME type of 'data\\금_따는_콩밭_김유정.pdf' is "cannot open `data\\\\352\\270\\210_\\353\\224\\260\\353\\212\\224_\\354\\275\\251\\353\\260\\255_\\352\\271\\200\\354\\234\\240\\354\\240\\225.pdf' (illegal byte sequence)". This file type is not currently supported in unstructured.


The MIME type of 'data\\동백꽃_김유정.pdf' is "cannot open `data\\\\353\\217\\231\\353\\260\\261\\352\\275\\203_\\352\\271\\200\\354\\234\\240\\354\\240\\225.pdf' (illegal byte sequence)". This file type is not currently supported in unstructured.
The MIME type of 'data\\메밀꽃_필_무렵_이효석.pdf' is "cannot open `data\\\\353\\251\\224\\353\\260\\200\\352\\275\\203_\\355\\225\\204_\\353\\254\\264\\353\\240\\265_\\354\\235\\264\\355\\232\\250\\354\\204\\235.pdf' (illegal byte sequence)". This file type is not currently supported in unstructured.
The MIME type of 'data\\배따라기_김동인.pdf' is "cannot open `data\\\\353\\260\\260\\353\\224\\260\\353\\235\\274\\352\\270\\260_\\352\\271\\200\\353\\217\\231\\354\\235\\270.pdf' (illegal byte sequence)". This file type is not currently supported in unstructured.
The MIME type of 'data\\백치_아다다_계용묵.pdf' is "cannot open `data\\\\353\\260\\261\\354\\271\\230_\\354\\225\\204\\353\\213\\244\\353\\213\\244_\\352\\263\\204\\354\\232\\251\\353\\254\\265.pdf' (illegal byte s

In [19]:

# 메타데이터 추출 함수
def extract_metadata(pdf_path):
    # 여기에 소설 제목과 저자 추출 로직을 추가하세요.
    # 예제: 파일명에서 제목과 저자 추출
    filename = os.path.splitext(os.path.basename(pdf_path))[0]
    parts = filename.split("_")
    title = parts[0] if len(parts) > 0 else "Unknown Title"
    author = parts[1] if len(parts) > 1 else "Unknown Author"
    return title, author

# 전체 텍스트 가져오기 함수
def get_full_text(documents):
    return "".join([doc.page_content for doc in documents])

In [ ]:
# 3. OpenAIEmbeddings 객체 생성 -> embedding vector를 생성.
from langchain_openai import OpenAIEmbeddings

embedding_model_name = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(
    model=embedding_model_name)

embedding_docs = embedding_model.embed_documents(--)
print(len(embedding_docs))

TypeError: argument 'text': 'Document' object cannot be converted to 'PyString'

In [ ]:
# PINECONE_API_KEY를 활용한 vector store
from pinecone import Pinecone, ServerlessSpec
import os
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")   



In [ ]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

COLLECTION_NAME = "RAG_novel"
PERSIST_DIRECTORY = "vector_store/chroma/RAG_novel_db"  # 저장할 경로

# 저장하면서 생성
vector_store = Chroma.from_documents(
    documents=document_list,
    embedding=embedding_model,
    ids= ids,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)
vector_store._collection.count()

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

COLLECTION_NAME = "RAG_novel_db_openai"
PERSIST_DIRECTORY = "vector_store/chroma/RAG_novel_db"

EMBEDDING_MODEL_NAME = "text-embedding-3-small"
embedding_model = OpenAIEmbeddings(model=EMBEDDING_MODEL_NAME)

vector_store = Chroma(
    embedding_function=embedding_model,
    collection_name=COLLECTION_NAME,
    persist_directory=PERSIST_DIRECTORY
)
retriever = vector_store.as_retriever()
query = ""
result = retriever.invoke(query)

- app.py
    - steamlit을 이용해서 서비스하는 application
    - vector_store.ipynb에서 구축한 DB를 이용해 질문과 관련된 내용들을 조회해서 LLM에 전송하고 그 결과를 채팅창에 출력.
    - memory 기능은 사용하지 않는다.

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings import OpenAIEmbeddings
import pinecone

# Pinecone API 초기화
pinecone.init(api_key="your-pinecone-api-key", environment="your-pinecone-environment")

# 데이터 폴더 설정
data_folder = "data/"
file_paths = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.endswith(".pdf")]

# 텍스트 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # 청크 크기 설정
    chunk_overlap=100  # 청크 간 중복 설정
)

# 메타데이터 추출 함수
def extract_metadata(pdf_path):
    # 파일명에서 제목과 저자 추출 (예: "소설제목_저자.pdf")
    filename = os.path.splitext(os.path.basename(pdf_path))[0]
    parts = filename.split("_")
    title = parts[0] if len(parts) > 0 else "Unknown Title"
    author = parts[1] if len(parts) > 1 else "Unknown Author"
    return title, author

# 전체 텍스트 가져오기 함수
def get_full_text(documents):
    return "".join([doc.page_content for doc in documents])

# 문서 처리 및 벡터 스토어에 저장
all_split_docs = []
for file_path in file_paths:
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    title, author = extract_metadata(file_path)

    split_docs = text_splitter.split_documents(documents)
    full_text = get_full_text(documents)
    for doc in split_docs:
        doc.metadata.update({
            "title": title,
            "author": author,
            "full_text": full_text  # 소설 전체 내용
        })
    all_split_docs.extend(split_docs)

# 벡터 스토어 생성 - Chroma
chroma_store = Chroma(
    collection_name="novels",
    embedding_function=OpenAIEmbeddings()
)
chroma_store.add_documents(all_split_docs)

# 벡터 스토어 생성 - Pinecone
pinecone_index_name = "novels-index"
if pinecone_index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=pinecone_index_name,
        dimension=OpenAIEmbeddings().dimension
    )
pinecone_store = Pinecone(
    index=pinecone.Index(pinecone_index_name),
    embedding_function=OpenAIEmbeddings()
)
pinecone_store.add_documents(all_split_docs)

print("Vector store 저장 완료!")
